In [ ]:
# Install dependencies silently
%pip install pyautogen autogen google-generativeai --quiet


In [ ]:
import os
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

In [ ]:
# Setup Gemini API
os.environ["GOOGLE_API_KEY"] = "Your_key"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

llm_setup = {
    "config_list": [
        {
            "model": "gemini-1.5-flash",
            "api_key": os.environ["GOOGLE_API_KEY"],
            "api_type": "google"
        }
    ]
}
 

In [ ]:
# Strategic Evaluation Engine
class StrategyEngine:
    def __init__(self):
        self.strategy = None

    def evaluate(self, data):
        net_investments = data['fixed_deposits'] + data['sip_value'] + data['property_value']
        if net_investments < data['income'] * 2:
            self.strategy = "GROWTH"
        else:
            self.strategy = "VALUE"
        return self.strategy


In [ ]:
# Define Financial Agents
analysis_bot = AssistantAgent(
    name="AnalysisBot",
    llm_config=llm_setup,
    system_message="""You are AnalysisBot.
    Your task is to analyze the user's portfolio and identify the suitable investment category: Growth or Value.

    Your response must include:
    - Portfolio review
    - Recommended investment type
    - Rationale for your decision"""
)

growth_specialist = AssistantAgent(
    name="GrowthSpecialist",
    llm_config=llm_setup,
    system_message="""You are GrowthSpecialist.
    Recommend aggressive investment opportunities such as:
    - High-return mutual funds
    - Emerging market equities
    - Startups or high-volatility assets

    Mention expected returns clearly."""
)

value_specialist = AssistantAgent(
    name="ValueSpecialist",
    llm_config=llm_setup,
    system_message="""You are ValueSpecialist.
    Recommend steady investment options like:
    - Blue-chip stocks
    - Government bonds
    - Dividend-generating funds or deposits

    Focus on risk-free, long-term value investments."""
)

final_reporter = AssistantAgent(
    name="ReportGenerator",
    llm_config=llm_setup,
    system_message="""You are ReportGenerator.
    Compile a comprehensive investment report combining:
    - Analysis insights
    - Recommendations from investment agents

    Present the output in a professional, user-friendly tone with clear suggestions."""
)

client_proxy = UserProxyAgent(
    name="ClientProxyAgent",
    human_input_mode="NEVER",
    system_message="You represent the investor and start the evaluation process."
)


In [ ]:
# Portfolio Management Function
def start_investment_planner():
    print("💼 Welcome to AI Investment Planner")

    # Collect user portfolio inputs
    income = float(input("Enter your monthly income: ₹"))
    fixed = float(input("Amount in Fixed Deposits: ₹"))
    sip = float(input("Total SIP investments: ₹"))
    property_val = float(input("Real Estate asset value: ₹"))

    user_data = {
        "income": income,
        "fixed_deposits": fixed,
        "sip_value": sip,
        "property_value": property_val
    }

    # Evaluate the strategy
    strategy_checker = StrategyEngine()
    investment_style = strategy_checker.evaluate(user_data)
    print(f"\n🔍 Recommended Strategy: {investment_style}")

    # Select relevant investment agent
    selected_advisor = growth_specialist if investment_style == "GROWTH" else value_specialist

    # Group Chat setup
    chat_room = GroupChat(
        agents=[client_proxy, analysis_bot, selected_advisor, final_reporter],
        messages=[],
        max_round=6
    )

    group_manager = GroupChatManager(
        groupchat=chat_room,
        llm_config=llm_setup
    )

    # Initiate conversation
    portfolio_summary = f"""
    Investor Data:
    - Monthly Income: ₹{income:,.2f}
    - Fixed Deposits: ₹{fixed:,.2f}
    - SIPs: ₹{sip:,.2f}
    - Real Estate Value: ₹{property_val:,.2f}

    Investment Strategy Selected: {investment_style}

    AnalysisBot: Please review the investor profile.
    {selected_advisor.name}: Recommend suitable {investment_style.lower()} investments.
    ReportGenerator: Summarize a detailed financial report.
    """

    print("\n🚀 Launching AI Investment Session...\n")
    result = client_proxy.initiate_chat(
        recipient=group_manager,
        message=portfolio_summary,
        max_turns=6
    )

    # Final Display
    print("\n✅ Investment Planning Complete")
    print(f"📈 Strategy Used: {investment_style}")
    print(f"💰 Total Assets: ₹{fixed + sip + property_val:,.2f}")


In [ ]:
# Run the system
if __name__ == "__main__":
    start_investment_planner()
